# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на вопросы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?



## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # импорт библиотеки Pandas

In [2]:
df = pd.read_csv('/datasets/data.csv')  # чтение файла и сохранение в переменной df

In [3]:
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() # получаем общую информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Типы данных - `int64`, `float64`, `object`.

Cогласно документации к данным:
* `children` - количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.




**Вывод**

В каждой стоке таблицы - информация о клиентах. Часть колонок предоставляет информацию о самом клиенте: количество детей, трудовой стаж, возраст, уровень образования, семейное положение, пол, тип занятости, наличие задолжности по возврату кредита, ежемесячный доход и цель получения кредита. В остальных колонках идентификаторы(`id`). 

Предварительно можно утверждать, что даных достаточно для ответа на вопросы. Но в данных встречаются пропуски, их нужно устранить.



## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
df.isna().sum() # подсчет пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Не все пропущеные значения влияют на исследование. Пропуски в `days_employed` неважны для нашего исследования.

Но пропуски в `total_income` повлияют нарезультаты исследования, `total_income` количественная переменная, а значит мы можем востановить данные применив среднее арифметическое или медиану. В данном случае целесобразно использовать медианное значение т.к. заработная плата у клиентов может сильно отличаться.

In [6]:
df['days_employed'] = df['days_employed'].fillna(0) #заменим пропуски в столбце 'days_employed' на 0

Определим какие типы занятости встречаются в столбце `income_type`. 

In [7]:
df['income_type'].unique() # находим уникальные категории в столбце 'income_type'

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [8]:
df.groupby('income_type')['total_income'].median() # определение медианной ЗП по разным категориям

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполнения пропусков в стоблце `total_income` для категории 'безработный'

In [9]:
# медианная ЗП для категории 'безработный'
unemployed_income_median = df[df['income_type'] == 'безработный']['total_income'].median()

In [10]:
# заполнения пропусков в стоблце total_income для категории 'безработный'
df.loc[df['income_type'] == 'безработный', 'total_income'] = df.loc[df['income_type'] == 'безработный', 'total_income'].fillna(unemployed_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'в декрете'

In [11]:
# медианная ЗП для категории 'в декрете'
decree_income_median = df[df['income_type'] == 'в декрете']['total_income'].median()

In [12]:
# заполнения пропусков в стоблце total_income для категории 'в декрете'
df.loc[df['income_type'] == 'в декрете', 'total_income'] = df.loc[df['income_type'] == 'в декрете', 'total_income'].fillna(decree_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'госслужащий'

In [13]:
# медианная ЗП для категории 'госслужащий'
state_income_median = df[df['income_type'] == 'госслужащий']['total_income'].median()

In [14]:
# заполнения пропусков в стоблце total_income для категории 'госслужащий'
df.loc[df['income_type'] == 'госслужащий', 'total_income'] = df.loc[df['income_type'] == 'госслужащий', 'total_income'].fillna(state_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'компаньон'

In [15]:
# медианная ЗП для категории 'компаньон'
companion_income_median = df[df['income_type'] == 'компаньон']['total_income'].median()

In [16]:
# заполнения пропусков в стоблце total_income для категории 'компаньон'
df.loc[df['income_type'] == 'компаньон', 'total_income'] = df.loc[df['income_type'] == 'компаньон', 'total_income'].fillna(companion_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'пенсионер'

In [17]:
# медианная ЗП для категории 'пенсионер'
pensioner_income_median = df[df['income_type'] == 'пенсионер']['total_income'].median()

In [18]:
# заполнения пропусков в стоблце total_income для категории 'компаньон'
df.loc[df['income_type'] == 'пенсионер', 'total_income'] = df.loc[df['income_type'] == 'пенсионер', 'total_income'].fillna(pensioner_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'предприниматель'

In [19]:
# медианная ЗП для категории 'предприниматель'
businessman_income_median = df[df['income_type'] == 'предприниматель']['total_income'].median()

In [20]:
# заполнения пропусков в стоблце total_income для категории 'предприниматель'
df.loc[df['income_type'] == 'предприниматель', 'total_income'] = df.loc[df['income_type'] == 'предприниматель', 'total_income'].fillna(businessman_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'сотрудник'

In [21]:
# медианная ЗП для категории 'сотрудник'
employee_income_median = df[df['income_type'] == 'сотрудник']['total_income'].median()

In [22]:
# Заполнения пропусков в стоблце total_income для категории 'сотрудник'
df.loc[df['income_type'] == 'сотрудник', 'total_income'] = df.loc[df['income_type'] == 'сотрудник', 'total_income'].fillna(employee_income_median)

Заполнения пропусков в стоблце `total_income` для категории 'студент'

In [23]:
# медианная ЗП для категории 'студент'
student_income_median = df[df['income_type'] == 'студент']['total_income'].median()

In [24]:
# Заполнения пропусков в стоблце total_income для категории 'студент'
df.loc[df['income_type'] == 'студент', 'total_income'] = df.loc[df['income_type'] == 'студент', 'total_income'].fillna(student_income_median)

Убедимся в отсутсвии пропусков:

In [25]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В столбцах `days_employed` и `total_income` были обнаружены пропуски. Предположим, что это ошибка при заполнении или ошибка программы банка т.к входные данные это статистика платёжеспособности, а в таких данных информация о трудовом стаже и доходе отсутствовать не может.


Пропущеные значения обработаны, можем приступать к следующему этапу предобработки данных.

### Замена типа данных

В колонках `days_employed` и `total_income` изменим вещественный тип данных на целочисленный c помощью функции `astype()`.

In [26]:
df['days_employed'] = df['days_employed'].astype(int) # замена типа данных на int

In [27]:
df['total_income'] = df['total_income'].astype(int) # замена типа данных на int

Посмотрим общую информацию о таблице и убедимся в том, что тип данных изменён:

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

В колонках `days_employed` и `total_income` изменили вещественный тип данных на целочисленный, применили `astype()` т.к эта функция даёт нам возможность выбрать необходимый тип данных. Работать с целочисленными значениями нам будет удобнее, при этом `int` весит меньше чем `float`.

### Обработка дубликатов

При ознакомлении с таблицей я заметил, что в столбце `education` данные заполнены в разных регистрах. 

Для того чтобы обнаружить все дубликаты приведем данные этого столбца к нижнему регистру:

In [29]:
df['education'] = df['education'].str.lower()  # приведение столбца 'education' к нижнему регистру

Приведем значения столбца `days_employed` к абсолютным значениям т.к трудовой стаж не может быть отрицательным.

In [30]:
df['days_employed'] = df['days_employed'].apply(abs) # приведение столбца 'days_employed' к абсолютным значениям

In [ ]:
df['days_employed'].abs()

Вызовем метод для поиска явных дубликатов и посчитаем их количество:

In [31]:
df.duplicated().sum() # подсчет дубликатов

71

Воспользуемся методом `drop_duplicates()` для удаления дубликатов и перезапишем индексы датафрейма с помощью `reset_index(drop=True)`:

In [32]:
df  = df.drop_duplicates().reset_index(drop=True) # удаление дубликатов

Еще раз посчитаем явные дубликаты и убедимся, что мы от них полностью избавились:

In [33]:
df.duplicated().sum()

0

Посмотрим уникальные значения в необходимых для исследования столбцах для поиска артефактов в данных:

In [34]:
df['children'].value_counts() # вывод уникальных значений столбца 'children'

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Колличество детей в семье не может быть отрицательным, предположим что это ошибка данных и сделаем все значения положительными с помощью функции `abs()`. Так же обратим вниманиие на клиентов у которых 20 детей, это маловероятно но вполне возможно. Из всего датасета таких клиентов 76, а это небольшой процент от данных, оставим эти строки без изменений.

In [35]:
df['children'] = df['children'].apply(abs) # приведения значений столбца 'children' к абсолютным

In [36]:
df['family_status'].value_counts() # вывод уникальных значений столбца 'family_status'

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В столбце `family_status` аномалий не обнаружено.

In [37]:
df['dob_years'].value_counts() # вывод уникальных значений столбца 'dob_years'

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [38]:
df['gender'].value_counts() # вывод уникальных значений столбца 'gender'

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

**Вывод**

Явные дубликаты в данных удалены.
Так же в данных есть необычные значения такие как `0` в стоблце `dob_years` и `XNA` в столбце `gender`, но для исследования эти столбцы нам скорее всего непригодятся и не повлияют на его результат, поэтому оставим их без изменения.


### Лемматизация

Импортируем библиотеку `pymystem3` необходимую для лемматизации:

In [39]:
from pymystem3 import Mystem # импорт библиотеки
m = Mystem()

Произведем лемматизацию столбца `purpose`:

In [40]:
lemmas = df['purpose'].apply(m.lemmatize) # лемматизация 

По умолчанию pymastem3 выдает список лемматизированых слов, применим метод `.join` для превращения списка в строку и сохраним всё это в новом столбце датасета.

In [41]:
df['lemmas'] = lemmas.apply(''.join) # добавление нового столбца с леммами

**Вывод**

С помощью `apply()` мы лемматизиривали весь столбец `purpose` и сохранили результат в новом столбце `lemmas`.

### Категоризация данных

Создадим словари для `education` и `family_status`, благодаря этому мы сможем обращаться к нужным строкам используя `id` категори.

In [42]:
df[['education', 'education_id']].drop_duplicates().reset_index(drop=True) # словарь для 'education'

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [43]:
df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True) # словарь для 'family_status'

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


В исследовании нам нужно найти зависимость между уровнем дохода и возврата кредита в срок.
C помощью метода `qcut()` получим 3 равные категории дохода.

Категории дохода:
* низкий - `20666 - 119261`
* средний - `119261 - 172357`
* высокий - `172357 - 2265604`

In [44]:
pd.qcut(df['total_income'], q = 3, precision = 0) # определим категории доходов 

0        (172357, 2265604]
1          (20666, 119261]
2         (119261, 172357]
3        (172357, 2265604]
4         (119261, 172357]
               ...        
21449    (172357, 2265604]
21450     (119261, 172357]
21451      (20666, 119261]
21452    (172357, 2265604]
21453      (20666, 119261]
Name: total_income, Length: 21454, dtype: category
Categories (3, interval[int64]): [(20666, 119261] < (119261, 172357] < (172357, 2265604]]

Напишем функцию, которая будет брать значение из столбца `total_income` и присваивать ему категорию:

In [45]:
def income_cat(row):
    if row < 119261:
        return 'низкий'
    if row < 172357:
        return 'средний'
    if row <= 2265604:
        return 'высокий'

С помощью метода `apply()` вызовем функцию `income_cat()` для всех строк, полученый результат сохраним в новом столбце `income_category`:

In [46]:
df['income_category'] = df['total_income'].apply(income_cat) # добавление нового столбца 'income_category'

Так же в исследовании нам нужно определить есть ли зависимость между наличием детей и возвратом кредита в срок.
Поэтому выделим новую категорию и создадим для нее столбец `children_category`.

Напишем функцию, которая будет брать значение из столбца `children` и определять наличие детей.

In [47]:
def children_cat(row):
    if row < 1:
        return 'yes'
    if row >= 1:
        return 'no'

Вызовем эту функцию для всех строк и сохраним результат в новом столбце:

In [48]:
# применение функции children_cat() и сохранение результата в новом столбце
df['children_category'] = df['children'].apply(children_cat) 

Теперь нам нужно определить категории целей кредита.

Для этого вызовем метод `unique()` и посмотрим уникальные значения в столбце с леммами целей.

In [49]:
df['lemmas'].unique()

array(['покупка жилье\n', 'приобретение автомобиль\n',
       'дополнительный образование\n', 'сыграть свадьба\n',
       'операция с жилье\n', 'образование\n', 'на проведение свадьба\n',
       'покупка жилье для семья\n', 'покупка недвижимость\n',
       'покупка коммерческий недвижимость\n',
       'покупка жилой недвижимость\n',
       'строительство собственный недвижимость\n', 'недвижимость\n',
       'строительство недвижимость\n',
       'на покупка подержать автомобиль\n',
       'на покупка свой автомобиль\n',
       'операция с коммерческий недвижимость\n',
       'строительство жилой недвижимость\n', 'жилье\n',
       'операция со свой недвижимость\n', 'автомобиль\n',
       'заниматься образование\n', 'сделка с подержанный автомобиль\n',
       'получение образование\n', 'свадьба\n',
       'получение дополнительный образование\n', 'покупка свой жилье\n',
       'операция с недвижимость\n', 'получение высокий образование\n',
       'свой автомобиль\n', 'сделка с автомобиль

Исходя из списка целей можно выделить следующие категории:

* недвижимость(жилье)
* автомобиль
* образование
* свадьба


Напишем функцию которая присвоит категории целям кредита по столбцу `lemmas`:

In [50]:
def purpose_category(row):
    if 'недвижимость' in row  or 'жилье' in row:
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'образование' in row:
        return 'образование'
    if 'свадьба' in row:
        return 'свадьба'

С помощью метода `apply()` вызовем функцию `purpose_category()` для всех строк, полученый результат сохраним в новом столбце `purpose_category`:

In [51]:
df['purpose_category'] = df['lemmas'].apply(purpose_category)

**Вывод**

Создали словари для быстрого обращения к строкам по `education_id` и `family_status_id`. 
Добавили три новых столбца в датасет с категориями дохода `income_category`, наличием детей `children_category` и категорией целей кредита `purpose_category`.

## Шаг 3. Ответьте на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Для ответа на этот вопрос создадим сводную таблицу, которая покажет клиентов без детей и с детьми, с разделением на должников и тех кто платил во вреемя:

In [52]:
pivot_children_debt = df.pivot_table(index=['children_category'], columns='debt', values='gender', aggfunc='count') # сводна таблица
pivot_children_debt.reset_index() # удалим мултииндекс

debt,children_category,0,1
0,no,6685,678
1,yes,13028,1063


Оределим долю клиентов которые `не имеют` детей и `не имели` задолжностей по кредитам:

In [53]:
no_children = pivot_children_debt.loc['no', 0]/(pivot_children_debt.loc['no',0] + pivot_children_debt.loc['no',1])
'Доля клиентов не имеющих детей и не имевших задолжностей по кредиту составляет: {:.0%}'.format(no_children)

'Доля клиентов не имеющих детей и не имевших задолжностей по кредиту составляет: 91%'

Оределим долю клиентов которые `имеют` детей и `не имели` задолжностей по кредитам:

In [54]:
yes_children = pivot_children_debt.loc['yes', 0]/(pivot_children_debt.loc['yes',0] + pivot_children_debt.loc['yes',1])
'Доля клиентов имеющих детей и не имевших задолжностей по кредиту составляет: {:.0%}'.format(yes_children)

'Доля клиентов имеющих детей и не имевших задолжностей по кредиту составляет: 92%'

**Вывод:**


Не зависимо от того есть у клиентов дети или нет, в 91% и 92% случаев они не имели задолжности по кредиту.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**


Для ответа на этот вопрос создадим сводную таблицу, которая покажет клиентов находящихся в разном семейном положении, с разделением на должников и тех кто платил во вреемя:

In [55]:
pivot_family_status = df.pivot_table(index = ['family_status_id','family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_family_status

,debt,0,1
family_status_id,family_status,,
0,женат / замужем,11408,931
1,гражданский брак,3763,388
2,вдовец / вдова,896,63
3,в разводе,1110,85
4,Не женат / не замужем,2536,274


Создал новый датафрейм т.к при извлечении нужных значений из сводной таблицы возникли проблемы :(

In [56]:
data = [[0, 11408, 931],
        [1, 3763, 388],
        [2, 896,63],
        [3,1110,85],
        [4, 2536, 247]]
columns = ['family_status_id', 'не должник', 'должник']
pivot_family_status = pd.DataFrame(data = data, columns = columns)
pivot_family_status

,family_status_id,не должник,должник
0,0,11408,931
1,1,3763,388
2,2,896,63
3,3,1110,85
4,4,2536,247


Определим долю клиентов находящихся в `официальном браке` и не имевших задолжностей по кредиту:

In [57]:
family_status_0 = pivot_family_status.loc[0,'не должник']/(pivot_family_status.loc[0,'не должник'] + pivot_family_status.loc[0,'должник'])
'Доля клиентов находящихся в официальном браке и не имевших задолжностей по кредиту составляет: {:.0%}'.format(family_status_0)

'Доля клиентов находящихся в официальном браке и не имевших задолжностей по кредиту составляет: 92%'

Определим долю клиентов находящихся в `гражданском браке` и не имевших задолжностей по кредиту:

In [58]:
family_status_1 = pivot_family_status.loc[1,'не должник']/(pivot_family_status.loc[1,'не должник'] + pivot_family_status.loc[1,'должник'])
'Доля клиентов находящихся в гражданском браке и не имевших задолжностей по кредиту составляет: {:.0%}'.format(family_status_1)

'Доля клиентов находящихся в гражданском браке и не имевших задолжностей по кредиту составляет: 91%'

Определим долю клиентов в статусе `вдовец/вдова` и не имевших задолжностей по кредиту:

In [59]:
family_status_2 = pivot_family_status.loc[2,'не должник']/(pivot_family_status.loc[2,'не должник'] + pivot_family_status.loc[2,'должник'])
'Доля клиентов в статусе вдовец/вдова и не имевших задолжностей по кредиту составляет: {:.0%}'.format(family_status_2)

'Доля клиентов в статусе вдовец/вдова и не имевших задолжностей по кредиту составляет: 93%'

Определим долю клиентов находящихся `в разводе` и не имевших задолжностей по кредиту:

In [60]:
family_status_3 = pivot_family_status.loc[3,'не должник']/(pivot_family_status.loc[3,'не должник'] + pivot_family_status.loc[3,'должник'])
'Доля клиентов находящихся в разводе и не имевших задолжностей по кредиту составляет: {:.0%}'.format(family_status_3)

'Доля клиентов находящихся в разводе и не имевших задолжностей по кредиту составляет: 93%'

Определим долю клиентов находящихся статусе `не женат/не замужем` и не имевших задолжностей по кредиту:

In [61]:
family_status_4 = pivot_family_status.loc[4,'не должник']/(pivot_family_status.loc[4,'не должник'] + pivot_family_status.loc[4,'должник'])
'Доля клиентов находящихся статусе не женат/не замужем и не имевших задолжностей по кредиту составляет: {:.0%}'.format(family_status_4)

'Доля клиентов находящихся статусе не женат/не замужем и не имевших задолжностей по кредиту составляет: 91%'

**Вывод:**

Не зависимо от семейного положения клиентов в 91-93% случаев они не имели задолжности по кредиту.

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на этот вопрос создадим сводную таблицу, которая покажет клиентов по  категориям дохода, с разделением на должников и тех кто платил во вреемя:

In [62]:
pivot_income_debt = df.pivot_table(index = 'income_category', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_income_debt.reset_index() # удалим мультииндекс

debt,income_category,0,1
0,высокий,6961,558
1,низкий,6570,581
2,средний,6182,602


Определим долю клиентов имеющих `высокий` доход и не имевших задолжностей по кредиту:

In [63]:
income_max = pivot_income_debt.loc['высокий', 0] / (pivot_income_debt.loc['высокий', 0] + pivot_income_debt.loc['высокий', 1])
'Доля клиентов имеющих высокий доход и не имевших задолжностей по кредиту составляет: {:.0%}'.format(income_max)

'Доля клиентов имеющих высокий доход и не имевших задолжностей по кредиту составляет: 93%'

Определим долю клиентов имеющих `низкий` доход и не имевших задолжностей по кредиту:

In [64]:
income_min = pivot_income_debt.loc['низкий', 0] / (pivot_income_debt.loc['низкий', 0] + pivot_income_debt.loc['низкий', 1])
'Доля клиентов имеющих низкий доход и не имевших задолжностей по кредиту составляет: {:.0%}'.format(income_min)

'Доля клиентов имеющих низкий доход и не имевших задолжностей по кредиту составляет: 92%'

Определим долю клиентов имеющих `средний` доход и не имевших задолжностей по кредиту:

In [65]:
income_mean = pivot_income_debt.loc['средний', 0] / (pivot_income_debt.loc['средний', 0] + pivot_income_debt.loc['средний', 1])
'Доля клиентов имеющих средний доход и не имевших задолжностей по кредиту составляет: {:.0%}'.format(income_mean)

'Доля клиентов имеющих средний доход и не имевших задолжностей по кредиту составляет: 91%'

**Вывод:**

Не зависимо от уровня доходов клиентов в 91-93% случаев они не имели задолжности по кредиту.

- **Как разные цели кредита влияют на его возврат в срок?**

Для ответа на этот вопрос создадим сводную таблицу, которая покажет клиентов по категориям целей кредита, с разделением на должников и тех кто платил во вреемя:

In [66]:
pivot_purpose_debt = df.pivot_table(index = 'purpose_category', columns = 'debt', values = 'gender', aggfunc = 'count')
pivot_purpose_debt.reset_index() # удалим мультииндекс

debt,purpose_category,0,1
0,автомобиль,3903,403
1,недвижимость,10029,782
2,образование,3643,370
3,свадьба,2138,186


Определим долю клиентов с целью кредита `'автомобиль'` и не имевших задолжностей по кредиту:

In [67]:
purpose_automobile = pivot_purpose_debt.loc['автомобиль', 0] / (pivot_purpose_debt.loc['автомобиль', 0] + pivot_purpose_debt.loc['автомобиль', 1])
'Доля клиентов c целью кредита автомобиль не имевших задолжностей по кредиту составляет: {:.0%}'.format(income_mean)

'Доля клиентов c целью кредита автомобиль не имевших задолжностей по кредиту составляет: 91%'

Определим долю клиентов с целью кредита `'недвижимость'` и не имевших задолжностей по кредиту:

In [68]:
purpose_immovables = pivot_purpose_debt.loc['недвижимость', 0] / (pivot_purpose_debt.loc['недвижимость', 0] + pivot_purpose_debt.loc['недвижимость', 1])
'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: {:.0%}'.format(purpose_immovables)

'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: 93%'

Определим долю клиентов с целью кредита `'образование'` и не имевших задолжностей по кредиту:

In [69]:
purpose_education = pivot_purpose_debt.loc['образование', 0] / (pivot_purpose_debt.loc['образование', 0] + pivot_purpose_debt.loc['образование', 1])
'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: {:.0%}'.format(purpose_education)

'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: 91%'

Определим долю клиентов с целью кредита `'свадьба'` и не имевших задолжностей по кредиту:

In [70]:
purpose_wedding = pivot_purpose_debt.loc['свадьба', 0] / (pivot_purpose_debt.loc['свадьба', 0] + pivot_purpose_debt.loc['свадьба', 1])
'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: {:.0%}'.format(purpose_wedding)

'Доля клиентов c целью кредита недвижимость не имевших задолжностей по кредиту составляет: 92%'

**Вывод:**

Не зависимо от целей кредита клиентов в 91-93% случаев они не имели задолжности по кредиту.

## Шаг 4. Общий вывод

**Ответив на все вопросы в исследовании можно сделать вывод:**

**Зависимости между наличием детей, семейным положением, уровнем дохода и возвратом кредита в срок нет. Так же разные цели кредита никак не влияют на его возврат в срок. Во всех случаях процент добросовестных клиентов находится в диапазоне 91-93%.**